In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

/Applications/Anaconda/anaconda3/envs/Python35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
import tensorflow as tf

tf.Session()

with tf.device('/cpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3])
# If you have gpu you can try this line to compute b with your GPU
#with tf.device('/gpu:0'):    
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2])
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

print(sess.run(c))

# Runs the op.
# Log information
options = tf.RunOptions(output_partition_graphs=True)
metadata = tf.RunMetadata()
c_val = sess.run(c, options=options, run_metadata=metadata)

print(metadata.partition_graphs)

sess.close()

[[22. 28.]
 [49. 64.]]
[node {
  name: "MatMul"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
          dim {
            size: 2
          }
        }
        tensor_content: "\000\000\260A\000\000\340A\000\000DB\000\000\200B"
      }
    }
  }
}
node {
  name: "_retval_MatMul_0_0"
  op: "_Retval"
  input: "MatMul"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "index"
    value {
      i: 0
    }
  }
}
library {
}
versions {
  producer: 26
}
]


In [26]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

tf.logging.set_verbosity(tf.logging.INFO)
sess = tf.InteractiveSession()

train = pd.read_csv('Train Data/train.csv')
print('Shape of the train data with all features:', train.shape)
train = train.select_dtypes(exclude=['object'])
print("")
print('Shape of the train data with numerical features:', train.shape)
train.drop('Observation',axis = 1, inplace = True)
train_labels = train['Energy']
print("train labels:",train_labels.shape)
train.fillna(0,inplace=True)
train.drop('Energy',axis = 1, inplace = True)
test = pd.read_csv('Train Data/test.csv')
test = test.select_dtypes(exclude=['object'])
#ID = test.Id
test.fillna(0,inplace=True)
test.drop('Observation',axis = 1, inplace = True)
test_labels = pd.read_csv('Train Data/sample_submission.csv')
test_labels.drop('Observation',axis = 1, inplace = True)
print("test_labels:",test_labels.columns)

print("")
print("List of features contained our dataset train:",list(train.columns))
print("List of features contained our dataset test:",list(test.columns))

Shape of the train data with all features: (15780, 26)

Shape of the train data with numerical features: (15780, 26)
train labels: (15780,)
test_labels: Index(['Energy'], dtype='object')

List of features contained our dataset train: ['T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint']
List of features contained our dataset test: ['T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint']


In [18]:
from sklearn.ensemble import IsolationForest

clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(train)
y_noano = clf.predict(train)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", train.shape[0])

Number of Outliers: 1580
Number of rows without outliers: 14200


In [19]:
train.head(10)

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,22.700000,37.200000,21.000000,38.000000,23.390000,37.290000,22.832857,34.942857,20.500000,42.826667,...,22.700000,45.360000,20.200000,38.663333,12.80,760.050000,62.000000,3.500000,28.0,5.65
1,21.500000,41.045000,20.500000,39.133333,22.926667,39.526667,21.700000,34.126667,18.633333,44.663333,...,22.926667,34.663333,19.730000,37.933333,8.47,764.166667,48.166667,8.000000,26.5,-1.92
2,21.666667,38.000000,22.600000,35.700000,21.890000,36.590000,22.000000,35.530000,19.000000,46.200000,...,21.890000,38.545000,19.790000,39.430000,10.60,757.600000,57.000000,2.000000,27.0,2.40
3,23.290000,38.530000,21.671429,38.000000,24.290000,36.200000,23.100000,35.090000,22.042857,43.625714,...,24.890000,38.600000,22.600000,36.194286,11.60,760.600000,63.000000,1.000000,40.0,4.67
4,24.000000,42.560000,26.830000,34.356000,26.530000,40.333333,24.500000,40.900000,21.500000,49.090000,...,23.390000,44.230000,22.600000,44.090000,17.70,756.983333,63.000000,1.000000,21.5,10.40
5,18.390000,43.363333,17.700000,42.433333,19.290000,42.500000,18.600000,41.663333,17.200000,55.360000,...,18.500000,47.600000,16.700000,46.000000,1.70,749.400000,97.000000,3.833333,32.5,1.25
6,23.000000,34.863333,21.000000,35.223333,23.000000,36.090000,22.633333,33.933333,21.390000,53.260000,...,24.100000,39.363333,20.318571,37.200000,8.00,765.950000,68.166667,1.000000,40.0,2.33
7,23.100000,35.000000,23.390000,31.856667,23.000000,35.290000,23.000000,33.200000,20.856667,38.363333,...,21.666667,37.863333,20.290000,36.700000,18.60,760.600000,38.000000,4.500000,40.0,4.05
8,21.000000,37.200000,18.500000,40.656667,20.390000,37.260000,21.600000,34.463333,20.093333,78.800000,...,21.000000,38.123333,17.290000,35.590000,4.67,748.833333,72.666667,3.666667,40.0,0.20
9,21.390000,41.850000,19.730000,41.560000,20.790000,41.433333,19.290000,37.790000,17.862222,49.565000,...,21.700000,43.221111,17.890000,39.200000,3.33,738.933333,88.666667,3.000000,64.0,1.60


In [24]:
import warnings
warnings.filterwarnings('ignore')

col_train = list(train.columns)
col_train_bis = list(train.columns)

#col_train_bis.remove('SalePrice')

mat_train = np.matrix(train)
mat_test  = np.matrix(test)
#mat_new = np.matrix(train.drop('SalePrice',axis = 1))
mat_y = np.array(train_labels).reshape((15780,1))

prepro_y = MinMaxScaler()
prepro_y.fit(mat_y)

prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_test)

train = pd.DataFrame(prepro.transform(mat_train),columns = col_train)
test  = pd.DataFrame(prepro_test.transform(mat_test),columns = col_train_bis)

train.head()

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,0.610504,0.247555,0.388439,0.493111,0.559930,0.310592,0.694286,0.272870,0.513774,0.121473,...,0.606097,0.551285,0.558653,0.396963,0.640288,0.715116,0.441176,0.250000,0.415385,0.597561
1,0.459244,0.358150,0.347977,0.524979,0.511498,0.431059,0.581000,0.236055,0.326094,0.151635,...,0.630293,0.177115,0.504065,0.366448,0.484532,0.810853,0.237745,0.571429,0.392308,0.228293
2,0.480252,0.270566,0.517919,0.428438,0.403136,0.272890,0.611000,0.299353,0.362960,0.176871,...,0.519630,0.312896,0.511034,0.429009,0.561151,0.658140,0.367647,0.142857,0.400000,0.439024
3,0.684874,0.285810,0.442775,0.493111,0.654007,0.251885,0.721000,0.279507,0.668898,0.134595,...,0.839877,0.314820,0.837398,0.293756,0.597122,0.727907,0.455882,0.071429,0.600000,0.549756
4,0.774370,0.401726,0.860231,0.390646,0.888153,0.474506,0.861000,0.541573,0.614317,0.224333,...,0.679753,0.511757,0.837398,0.623798,0.816547,0.643798,0.455882,0.071429,0.315385,0.829268


In [28]:
COLUMNS = col_train
FEATURES = col_train_bis
LABEL = "Energy"

# Columns for tensorflow
feature_cols = [tf.contrib.layers.real_valued_column(k) for k in FEATURES]

# Training set and Prediction set with the features to predict
training_set = train[COLUMNS]
#print(training_set)
prediction_set = train_labels
#print(prediction_set)
# Train and Test 
x_train, x_test, y_train, y_test = train_test_split(training_set[FEATURES] , prediction_set, test_size=0.33, random_state=42)
y_train = pd.DataFrame(y_train, columns = [LABEL])
training_set = pd.DataFrame(x_train, columns = FEATURES).merge(y_train, left_index = True, right_index = True)
training_set.head()

# Training for submission
training_sub = training_set[col_train]

In [29]:
y_test = pd.DataFrame(y_test, columns = [LABEL])
testing_set = pd.DataFrame(x_test, columns = FEATURES).merge(y_test, left_index = True, right_index = True)
testing_set.head()

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,...,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,Energy
14438,23.200000,41.200000,21.390000,41.900000,24.600000,39.663333,23.000000,40.50,22.1875,58.058750,...,44.316000,22.600000,43.290000,10.20,757.866667,86.000000,3.000000,40.0,7.97,70
9222,19.926667,45.863333,19.356667,44.400000,19.790000,44.900000,18.890000,46.43,17.1000,55.000000,...,48.590000,17.000000,45.290000,5.97,734.366667,91.333333,5.666667,40.0,4.63,230
9469,21.290000,41.590000,18.600000,44.590000,23.000000,39.590000,20.890000,39.40,20.3900,52.433333,...,39.700000,20.533333,42.933333,6.03,751.700000,87.000000,5.000000,27.0,3.97,50
11904,21.890000,44.133333,20.500000,43.230000,22.823333,41.790000,19.700000,42.29,19.7900,48.635000,...,47.045000,19.000000,44.590000,4.60,729.500000,99.000000,4.000000,28.0,4.40,120
2655,21.100000,45.863333,20.500000,44.933333,21.500000,42.590000,21.533333,44.79,18.7900,76.954000,...,51.826667,18.356667,47.060000,11.60,756.133333,88.166667,9.833333,40.0,9.58,80


In [30]:
tf.logging.set_verbosity(tf.logging.ERROR)
regressor = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols, 
                                          activation_fn = tf.nn.relu, hidden_units=[200, 100, 50, 25, 12])#,
                                         #optimizer = tf.train.GradientDescentOptimizer( learning_rate= 0.1 ))

In [31]:
training_set.reset_index(drop = True, inplace =True)

In [32]:
def input_fn(data_set, pred = False):
    
    if pred == False:
        
        feature_cols = {k: tf.constant(data_set[k].values) for k in FEATURES}
        labels = tf.constant(data_set[LABEL].values)
        
        return feature_cols, labels

    if pred == True:
        feature_cols = {k: tf.constant(data_set[k].values) for k in FEATURES}
        
        return feature_cols

In [33]:
regressor.fit(input_fn=lambda: input_fn(training_set), steps=2000)

DNNRegressor(params={'activation_fn': <function relu at 0x1047556a8>, 'hidden_units': [200, 100, 50, 25, 12], 'dropout': None, 'embedding_lr_multipliers': None, 'feature_columns': (_RealValuedColumn(column_name='T1', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='RH_1', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='T2', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='RH_2', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='T3', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='RH_3', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='T4', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='RH_4', dimension=1, default_value=None, dtype=t

In [34]:
ev = regressor.evaluate(input_fn=lambda: input_fn(testing_set), steps=1)

In [35]:
# Predictions
y = regressor.predict(input_fn=lambda: input_fn(testing_set))
predictions = list(itertools.islice(y, testing_set.shape[0]))

In [37]:
predictions = prepro_y.inverse_transform(np.array(predictions).reshape(5208,1))

In [40]:
reality = test_labels

In [44]:
y_predict = regressor.predict(input_fn=lambda: input_fn(test, pred = True))

def to_submit(pred_y,name_out):
    y_predict = list(itertools.islice(pred_y, test.shape[0]))
    y_predict = pd.DataFrame(prepro_y.inverse_transform(np.array(y_predict).reshape(len(y_predict),1)), columns = ['Energy'])
    #y_predict = y_predict.join(ID)
    y_predict.to_csv(name_out + '.csv',index=False)
    
to_submit(y_predict, "submission_continuous")

In [45]:
def leaky_relu(x):
    return tf.nn.relu(x) - 0.01 * tf.nn.relu(-x)

In [46]:
regressor = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols, 
                                          activation_fn = leaky_relu, hidden_units=[200, 100, 50, 25, 12])
    
# Deep Neural Network Regressor with the training set which contain the data split by train test split
regressor.fit(input_fn=lambda: input_fn(training_set), steps=2000)

# Evaluation on the test set created by train_test_split
ev = regressor.evaluate(input_fn=lambda: input_fn(testing_set), steps=1)

In [47]:
loss_score2 = ev["loss"]
print("Final Loss on the testing set with Leaky Relu: {0:f}".format(loss_score2))

Final Loss on the testing set with Leaky Relu: 8980.164062
